In [28]:
import numpy as np
from scipy.stats import norm

In [110]:
# loss matrix L_ij is the cost of guessing i when the truth is j
L = np.array([[1,1,1], [1,1,5], [1,5,1]])

# Transition matrix A_ij is the probabilty p(z_{t+1} = j|z_t = i)
# i.e. A_ij is the probability of going from i to j
K = 3
A = np.array([[0.3, 0.3, 0.4], [0.2, 0.5, 0.3], [0.1, 0.1, 0.8]])

# initial state probability
pi = [ 0.6, 0.3, 0.1 ]

# emission probabilities - assume Gaussian
u = [ 1, 4, 5 ]
sigma = [ 1, 1, 1.5 ]

In [111]:
np.argmax(np.random.multinomial(1, pi))
np.sum(A, axis=1)

array([1., 1., 1.])

In [112]:
# generate data
T = 3
def generate_HMM(T):
    ''' TODO: loss calibrated data generation'''
    X = np.zeros(T)
    Z = np.zeros(T)
    z = np.argmax(np.random.multinomial(1, pi))
    for t in np.arange(T):
        X[t] = np.random.normal(u[z], sigma[z])
        Z[t] = z
        z = np.argmax(np.random.multinomial(1, A[z,:]))
    return X,Z
X,Z = generate_HMM(T)

In [115]:
def gaussian_emission_prob(x, state):
    return norm.pdf(x, u[state], sigma[state])

emission_prob = gaussian_emission_prob

# loss calibrated Viterbi
# the element S_ij stores guessing i at time t with z_{t+1}=j
S_0 = np.zeros((K,K))

# for t in np.arange(T):

# time 0 (first step)
for k in np.arange(K): # iterate over possible guesses at time t
    for j in np.arange(K): # iterate over z_1
        for i in np.arange(K): # sum over z_0
            S_0[k,j] += L[k, i] * pi[i]* emission_prob(X[0], i) * A[i, j] * emission_prob(X[1], j)
            
# choose optimum guess z-hat at time 0 for each z_1
phi0 = np.argmin(S_0, axis=0)

# time 1
S_1 = np.zeros((K,K))
for k in np.arange(K): # iterate over possible guesses at time t
    for j in np.arange(K): # iterate over z_{t+1}
        for i in np.arange(K): # sum over z_t
            S_1[k,j] += L[k, i] * A[i, j] * emission_prob(X[2], j) * S_0[phi0[j], i]
            
# choose optimum guess z-hat at time 0 for each z at time 1
phi1 = np.argmin(S[1], axis=0)

# time 2 (last step)
S_2 = np.zeros((K,K))
for k in np.arange(K): # iterate over possible guesses at time t
    # no iteration over the next time step!
    for i in np.arange(K): # sum over z_t
        S_2[k,j] += L[k, i] * S_1[phi1[j], i]


In [116]:
print(X)
print(Z)
print(S_0)
print(S_1)
print(S_2)

[2.5972323  6.40258492 3.10673191]
[1. 2. 2.]
[[5.44728911e-09 9.60650545e-04 7.91207928e-03]
 [5.98746095e-09 1.02628702e-03 1.19648240e-02]
 [1.20036251e-08 2.95230731e-03 1.71352403e-02]]
[[4.26427131e-05 3.40385060e-04 7.93597366e-04]
 [1.79886246e-04 1.18759852e-03 3.82974720e-03]
 [7.59697490e-05 8.54710086e-04 9.31835949e-04]]
[[0.         0.         0.00519723]
 [0.         0.         0.02051622]
 [0.         0.         0.00994763]]


In [83]:
# return Viterbi path
back tracing easy?

array([[6.78403540e-22, 8.58681719e+01, 4.66199771e-02],
       [2.29617474e-22, 2.86697134e+01, 1.68132544e-02],
       [4.52402987e-22, 5.72472552e+01, 3.11289563e-02]])

In [54]:
np.arange(T)[1:]

array([1, 2, 3])